In [ ]:
#installing tweeter API
!pip install tweepy

In [ ]:
!pip install nltk
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#imports
import pandas as pd
import numpy as np
import tweepy
import re
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D

In [ ]:
#declaring constants
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
EMBEDDING_DIM = 300 #best suitable
EPOCHS= 10


In [ ]:
!pip install ftfy
import ftfy

     |████████████████████████████████| 64 kB 1.5 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=7aa3f4be4f50e6bcf66009643d4133834e6032fd03459db5ac5b9c31a98c61d0
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy


In [ ]:
#connecting to the api
auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
#mounting drive to avoid temporary storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#we say some invalid tweets from both 60 users as well as 100 users datasets
#initally we are working on removing the invalid data in 60 users and store all the data into new file
#code is as follows
d=data
for i in x:
  try:
    y=api.get_status(i).text

  except tweepy.TweepError as tw:

    k=data[data['id']==i].index.values
    d=d.drop(index=k)


In [ ]:
#copying data to file a
d.to_csv('a.csv')

In [ ]:
#similar operations are performed on 100 users to remove invalid data


In [ ]:
#adding a new column to a.csv for text of tweets
from csv import writer
from csv import reader
def add_column_in_csv(input_file, output_file, transform_row):
    """ Append a column in existing csv using csv.reader / csv.writer classes"""
    # Open the input_file in read mode and output_file in write mode
    with open(input_file, 'r') as read_obj, \
            open(output_file, 'w', newline='') as write_obj:
        # Create a csv.reader object from the input file object
        csv_reader = reader(read_obj)
        # Create a csv.writer object from the output file object
        csv_writer = writer(write_obj)
        # Read each row of the input csv file as list
        for row in csv_reader:
            # Pass the list / row in the transform function to add column text for this row
            transform_row(row, csv_reader.line_num)
            # Write the updated row / list to the output file
            csv_writer.writerow(row)

In [ ]:
input_file='/content/drive/MyDrive/valid.csv'
output_file='/content/drive/MyDrive/outputfinal2.csv'

header_of_new_col = 'Tweets'




# Add the column in csv file with header
add_column_in_csv(input_file,output_file,
                    lambda row, line_num: row.append(header_of_new_col)
                    if line_num == 1 else row.append(api.get_status(row[1]).text))

Preprocessing
Preprocessing the tweets in order to:

Remove links and images,
Remove hashtags,
Remove @ mentions,
Remove emojis,
Remove stop words,
Remove punctuation,
Get rid of stuff like "what's" and making it "what is',
Stem words so they are all the same tense (e.g. ran -> run)

In [ ]:
##expanding the contractions
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have",
  " u " : "you",
  " ur " :  "your",
  " n " : "and"
}

In [ ]:
final_data= pd.read_csv('/content/drive/MyDrive/Majorproject/final_output1.csv')
final_data


,Unnamed: 0,id,node,annotator1,annotator2,Tweets,final label
0,0,606626035476537344,821845,1,0,I've been a very lazy Tweeter lately.,1
1,1,594103782021189632,821845,0,0,"Good morning, beautiful people!",0
2,3,616222679557910528,821845,0,0,"Happy Canada Day, eh.",0
3,8,607680043112931328,821845,0,0,"I just got added to a Twitter list called ""Twi...",0
4,11,606136869512953856,821845,0,0,There's a webinar for everything these days. I...,0
...,...,...,...,...,...,...,...
5519,8677,605464565753614336,2400925915,0,0,Ty is one happy guy rn,0
5520,8682,587428261308661760,2400925915,0,0,Great weekend 🔴🔵⚫️,0
5521,8683,577141727045144576,2400925915,0,0,Them NNS had some cute chirps,0
5522,8684,564113332597047296,2400925915,0,0,"Good luck to the boys in cross lake tonight, G...",0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def expand(t):
    if type(t) is str:
      for key in contractions:
        value=contractions[key]
        t=t.replace(key,value)
      print (t)
    else:
      print(t)

In [ ]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        #if re.match("(\w+:\/\/\S+)", tweet) == None :#and len(tweet) > 10:
            #remove hashtag, @mention, emoji and image URLs
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())

            #fix weirdly encoded texts
        tweet = ftfy.fix_text(tweet)

            #expand contraction
        tweet = expandContractions(tweet)

            #remove punctuation
        tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            #stop words
        stop_words = set(stopwords.words('english'))
        word_tokens = nltk.word_tokenize(tweet)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        tweet = ' '.join(filtered_sentence)

            #stemming words
            #tweet = PorterStemmer().stem(tweet)
        lemmatizer = WordNetLemmatizer()
        tweet= lemmatizer.lemmatize(tweet)

        cleaned_tweets.append(tweet)

            #removing non english terms
        tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if w.lower() in words or not w.isalpha())

    return cleaned_tweets


In [ ]:
users_tweets = [z for z in final_data.iloc[0:,5]]
print(len(X_train))
X_60 =clean_tweets(X_train)


NameError: ignored

In [ ]:
print(len(X_60))


2762


In [ ]:
X_60[0]

'Welcome Vancouver'

Tokenizer.
Using a Tokenizer to assign indices and filtering out unfrequent words. Tokenizer creates a map of every unique word and an assigned index to it. The parameter called num_words indicates that we only care about the top 20000 most frequent words.



In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_60)

NameError: ignored

In [ ]:
#Applying the tokenizer to 60 users tweets.
sequences_60 = tokenizer.texts_to_sequences(X_60)


In [ ]:
#Number of unique words in tokenizer. Has to be <= 20,000
word_index = tokenizer.word_index
print(word_index)
print('Found %s unique tokens' % len(word_index))

{'i': 1, 'like': 2, 'the': 3, 'good': 4, 'people': 5, 'get': 6, 'one': 7, 'it': 8, 'today': 9, 'love': 10, 'day': 11, 'time': 12, 'know': 13, 'want': 14, 'would': 15, 'see': 16, 'go': 17, 'happy': 18, 'great': 19, 'co': 20, 'tonight': 21, 'new': 22, 'think': 23, 'you': 24, 'so': 25, 'if': 26, '2': 27, 'http': 28, 'going': 29, 'we': 30, 'right': 31, 'night': 32, 'need': 33, 'first': 34, '3': 35, 'best': 36, 'really': 37, 'much': 38, 'a': 39, 'life': 40, 'could': 41, 'ever': 42, 'my': 43, 'work': 44, 'us': 45, 'thanks': 46, 'someone': 47, 'way': 48, 'feel': 49, 'show': 50, 'and': 51, 'rt': 52, 'make': 53, 'morning': 54, 'last': 55, 'thing': 56, 'this': 57, 'got': 58, 'home': 59, '1': 60, 'better': 61, 'come': 62, 'game': 63, 'big': 64, 'back': 65, 'what': 66, 'just': 67, 'na': 68, 'things': 69, 'says': 70, 'anyone': 71, 'still': 72, 'year': 73, 'bad': 74, 'win': 75, 'getting': 76, 'watch': 77, 'hey': 78, 'tomorrow': 79, 'awesome': 80, 'also': 81, 'take': 82, 'say': 83, 'always': 84, 'nev

In [ ]:
format(sequences_60)

'[[226, 501], [71, 134, 329, 1010, 502, 261, 2864, 211, 330, 502, 103], [585, 1, 10], [503, 504, 395, 331, 104, 296], [78, 71, 49, 2, 672, 2865, 804, 805, 1796, 1797, 1311, 1798, 2866, 448], [2867, 1312, 2868, 2869, 1, 505, 32, 2870, 79, 396, 20, 2871], [90, 2872, 56, 199, 2873, 1, 91, 81, 199, 806], [1799, 2874, 2875], [57, 1800, 449, 2876, 2877, 2878], [1, 1313, 105, 141, 5, 673, 450, 1011, 451, 807, 808], [1, 506, 2879, 2880, 586, 507, 674, 106, 1801, 119, 1314, 5, 452, 34, 362], [587, 809, 397, 2881, 1802, 2882], [810, 227, 332, 2883], [1315, 1316, 1803, 212, 1, 15, 10, 16, 1317, 1804, 398, 2884, 125, 11], [1, 2885, 16, 1012, 105, 6, 588, 675, 245], [26, 42, 2886, 674, 2887, 1805, 50, 142, 21, 811, 2888, 2889, 4, 143], [1, 2, 228, 1013, 5, 6, 1318, 1, 10, 228, 676, 6, 1318, 297, 229, 1319, 2890], [1, 399, 92, 1320, 104, 589, 115, 1, 261, 1014, 812, 298], [1015, 1016, 333, 453, 104, 2891, 813, 332, 1, 1806, 332, 1017, 814], [2892, 2893, 1807, 1321, 35], [3, 677, 135, 1018, 7, 61, 69

In [ ]:
#Pad sequences all to the same length of 140 words.
data_60 = pad_sequences(sequences_60, maxlen=140)
y=final_data.iloc[:,6]
y=np.asarray(y_train)
print(data_60)
print('Shape of data_d tensor:', data_60.shape)
print('shape of label sensor', y.shape)

[[   0    0    0 ...    0  226  501]
 [   0    0    0 ...  330  502  103]
 [   0    0    0 ...  585    1   10]
 ...
 [   0    0    0 ... 7326 1784  952]
 [   0    0    0 ... 1010 7328 2861]
 [   0    0    0 ...  608  294 7329]]
Shape of data_d tensor: (2762, 140)
shape of label sensor (2762,)


In [ ]:
data_train

array([ 211, 2419,  179, ..., 1972, 1816, 1917])

In [ ]:
data_train[0]

280

Embedding Matrix
The embedding matrix is a n x m matrix where n is the number of words and m is the dimension of the embedding. In this case, m=300 and n=11096. We take the min between the number of unique words in our tokenizer and max words in case there are less unique words than the max we specified. Length 100–300 dimensions. This range of vector sizes for the embeddings were shown in the glove paper to be the range with the most useful results.

In [ ]:
#Load Pretrained Word2Vec Model
EMBEDDING_FILE= '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
#Embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index))
nb_words=nb_words+1
skipped_words=0


embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, idx in word_index.items():
  try:
    embedding_vector=word2vec [word]
  except:
    skipped_words=skipped_words+1
    pass
    #if word in word2vec.vocab and idx < nb_words :
        #embedding_matrix[idx] = word2vec.word_vec(word)
  if embedding_vector is not None:
    embedding_matrix[idx]=embedding_vector
print(embedding_matrix.shape)

(7330, 300)


In [ ]:
print(embedding_matrix)
print("shape",embedding_matrix.shape)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.22558594 -0.01953125  0.09082031 ...  0.02819824 -0.17773438
  -0.00604248]
 [ 0.10351562  0.13769531 -0.00297546 ...  0.04394531 -0.14550781
   0.07128906]
 ...
 [-0.05712891  0.37304688  0.09570312 ...  0.0534668  -0.06445312
   0.33398438]
 [-0.24414062  0.19238281  0.08935547 ...  0.17480469 -0.08789062
  -0.27734375]
 [ 0.18652344  0.12597656 -0.03491211 ... -0.10107422  0.14257812
  -0.37890625]]
shape (7330, 300)


In [ ]:
embedding_matrix[3]-word2vec.wv['the']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

Splitting and Formatting Data.
60 users is spilt into both testing and traing in a ranfom format

In [ ]:
final_label= final_data.iloc[0:,6].values
count=0
for i in final_label:
  if i==1:
    count=count+1


In [ ]:
count

520

In [ ]:
X= final_data['Tweets']
y=final_data['final label']


In [ ]:
# create dataset
#X, y = make_classification(n_samples=5522, weights=[0.906], flip_y=0, random_state=1)
#print(Counter(y))


In [ ]:
count1=0
for i in y:
  if y[i]==0:
    count1=count1+1

In [ ]:
count1

520

In [ ]:
# split into train test sets
X1, X2, y1, y2 = train_test_split(X, y, test_size=0.50, random_state=1)
print(Counter(y1))
print(Counter(y2))

Counter({0: 2479, 1: 283})
Counter({0: 2525, 1: 237})


In [ ]:
print(len(X1))
print(len(y1))

2762
2762


In [ ]:
X_train=np.array(X1)
y_train=np.array(y1)


In [ ]:
X_test=np.array(X2)
y_test=np.array(y2)

In [ ]:
count=0
for i in y_test:
  if y_test[i]==1:
   count=count+1

In [ ]:
count

0

Building RNN Model

In [ ]:
#create the embedding layer
embedding_layer=Embedding(nb_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False)



In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
#define model
model=Sequential()
model.add(embedding_layer)
model.add(SimpleRNN(128,activation='relu',return_sequences=True))
model.add(SimpleRNN(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 140, 300)          2199000   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 140, 128)          54912     
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 256)               98560     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2,352,729
Trainable params: 153,729
Non-trainable params: 2,199,000
_________________________________________________________________
None


Compiling

In [ ]:
from keras.optimizers import *
#compile network
# = SGD(lr=0.001, decay=1e-6)
model.compile(loss="binary_crossentropy",metrics=['accuracy'])


In [ ]:
data_60.shape

(2762, 140)

In [ ]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_train[3565]

1

In [ ]:
X_train[3225]

'Welcome to Vancouver @conhockey19 !!'

In [ ]:
y_train.shape

(2762,)

In [ ]:
#fit model

model.fit(data_60,y_train,epochs=1,verbose=1)

NameError: ignored

Preprocessing testing dataset

In [ ]:
test_set =clean_tweets(X_test)

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(test_set)
#Applying the tokenizer to 60 users tweets.
sequences_test = tokenizer.texts_to_sequences(test_set)
test_60 = pad_sequences(sequences_60, maxlen=MAX_SEQUENCE_LENGTH)



In [ ]:
X_test

array(['WCS Stars Choir with a stellar performance at @Telemiracle this morning. Making Weyburn proud.',
       'My wildflowers are blooming 😍',
       'Dear @Shedoesthecity, I LOVE ALL THESE SHOWS! Help me hibernate this winter with @HBOCanada gift set. #HBOHoliday2015',
       ..., 'Wanderlust- the desire to travel',
       '"Leadership is the art of getting someone else to do something you want done because he wants to do it." ~ Dwight D. Eisenhower',
       'Phew! I am on week 2 of #PiYo and loving it! I cannot wait to see the results in 6 weeks time~ #workinghard #beachbody'],
      dtype=object)

In [ ]:
pred=model.predict(test_60, batch_size=40)

In [ ]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
labels_pred = np.round(pred.flatten())
accuracy = accuracy_score(y_test, labels_pred)
print("Accuracy: %.2f%%" % (accuracy*100))

Accuracy: 91.42%


Building Model (LSTM + CNN)
The model takes in an input and then outputs a single number representing the probability that the tweet indicates depression. The model takes in each input sentence, replace it with it's embeddings, then run the new embedding vector through a convolutional layer. CNNs are excellent at learning spatial structure from data, the convolutional layer takes advantage of that and learn some structure from the sequential data then pass into a standard LSTM layer. Last but not least, the output of the LSTM layer is fed into a standard Dense model for prediction.

In [ ]:
model1 = Sequential()
# Embedded layer
model1.add(Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH, trainable=False))
# Convolutional Layer
model1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model1.add(MaxPooling1D(pool_size=2,strides=None, padding="valid", data_format="channels_last"))
model1.add(Dropout(0.2))
# LSTM Layer
model1.add(LSTM(300))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))

In [ ]:
model1.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 140, 300)          2199000   
                                                                 
 conv1d (Conv1D)             (None, 140, 32)           28832     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 32)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 70, 32)            0         
                                                                 
 lstm (LSTM)                 (None, 300)               399600    
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                      

Training the Model
The model is trained EPOCHS time, and Early Stopping argument is used to end training if the loss or accuracy don't improve within 3 epochs.

In [ ]:
#early_stop = EarlyStopping(monitor='val_loss', patience=3)

model1.fit(data_60, y_train ,epochs=1)


87/87 [==============================] - 27s 277ms/step - loss: 0.3476 - acc: 0.8921


In [ ]:
pred1=model1.predict(test_60 , batch_size=40)

In [ ]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
labels_pred1 = np.round(pred1.flatten())
accuracy = accuracy_score(y_test, labels_pred1)
print("Accuracy: %.2f%%" % (accuracy*100))

Accuracy: 91.42%
